## Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from glob import glob
from tqdm.auto import tqdm 

from sklearn.preprocessing import RobustScaler
import xgboost as xgb
from xgboost import XGBRegressor

import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score

import joblib
from datetime import datetime, timedelta
from geocube.api.core import make_geocube
import geopandas as gpd
from shapely import wkt
from shapely.geometry import mapping
import xarray as xr
import cartopy.crs as ccrs
from statsmodels.nonparametric.smoothers_lowess import lowess
import shap

import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = 'Times New Roman'

# Define directories
WORK_DIR = "/beegfs/halder/SIMPLACE_WDIR/workshop/"

## Read the data

In [ ]:
# Reading the data as a pandas dataframe
data_path = os.path.join(WORK_DIR, "data", "data.csv")
data = pd.read_csv(data_path)
print(data.shape)
data.head()

### Variables description


| Variable                                         | Description                                                               | Unit / Notes          |
| ------------------------------------------------ | ------------------------------------------------------------------------- | --------------------- |
| **NUTS\_ID**                                     | Regional identifier (NUTS3 level code for EU regions)                     | Categorical           |
| **Harvest\_Year**                                | Harvest year of the crop                                                  | Year (YYYY)           |
| **Yield\_t\_ha**                                 | Simulated crop yield by Simplace                                          | tons per hectare      |
| **Yield\_Obs**                                   | Observed crop yield (reference/ground-truth)                              | tons per hectare      |
| **crop**                                         | Crop type (e.g., maize, wheat)                                            | Categorical           |
| **Rain\_avg\_pX**                                | Average rainfall during phenological stage *pX*                           | mm                    |
| **Rain\_1std\_pX**                               | Count of days rainfall exceeded ±1σ deviation from mean during stage *pX* | count                 |
| **Rain\_2std\_pX**                               | Count of days rainfall exceeded ±2σ deviation                             | count                 |
| **Radiation\_avg\_pX**                           | Average solar radiation during stage *pX*                                 | MJ m⁻² day⁻¹          |
| **Radiation\_1std\_pX**, **Radiation\_2std\_pX** | Extreme event counts for solar radiation                                  | count                 |
| **Tmax\_avg\_pX**                                | Average maximum daily temperature during stage *pX*                       | °C                    |
| **Tmax\_1std\_pX**, **Tmax\_2std\_pX**           | Extreme high/low temperature event counts                                 | count                 |
| **Tmin\_avg\_pX**                                | Average minimum daily temperature during stage *pX*                       | °C                    |
| **Tmin\_1std\_pX**, **Tmin\_2std\_pX**           | Extreme low temperature event counts                                      | count                 |
| **ClimateWaterBalance\_avg\_pX**                 | Average rainfall – potential evapotranspiration (water availability)      | mm                    |
| **TotalSoilAvailWaterContent\_avg\_pX**          | Mean soil water available for crops during stage *pX*                     | mm                    |
| **TotalSoilAvailWaterContent\_1std/2std\_pX**    | Extreme soil water condition event counts                                 | count                 |
| **LAI\_max\_pX**                                 | Maximum Leaf Area Index during stage *pX*                                 | m² leaf per m² ground |
| **AGBiomass\_t\_ha\_max\_pX**                    | Maximum aboveground biomass during stage *pX*                             | tons per hectare      |
| **ActualEvapotranspiration\_max\_pX**            | Maximum actual evapotranspiration during stage *pX*                       | mm                    |
| **maxSeminal\_RootDepth**                        | Maximum root depth attained                                               | cm                    |
| **TRANRF**                                       | Ratio of actual to potential transpiration (water stress index)           | dimensionless         |
| **NNI**                                          | Nitrogen Nutrition Index                                                  | dimensionless         |
| **PNI**                                          | Phosphorus Nutrition Index                                                | dimensionless         |
| **KNI**                                          | Potassium Nutrition Index                                                 | dimensionless         |
| **NPKI**                                         | Combined NPK index (nutrient adequacy)                                    | dimensionless         |
| **NUptake\_SlimN**                               | Nitrogen uptake from soil mineral N pool                                  | kg N ha⁻¹             |
| **cropNuptake\_kg\_ha**                          | Crop nitrogen uptake                                                      | kg N ha⁻¹             |
| **cropPuptake\_kg\_ha**                          | Crop phosphorus uptake                                                    | kg P ha⁻¹             |
| **TotalUptakeP**                                 | Total phosphorus uptake                                                   | kg P ha⁻¹             |
| **CumulatedFertilizerP**                         | Cumulative phosphorus fertilizer input                                    | kg P ha⁻¹             |
| **inputChemN\_kg\_ha**                           | Applied chemical nitrogen fertilizer                                      | kg N ha⁻¹             |
| **PLeaching**                                    | Phosphorus lost via leaching                                              | kg P ha⁻¹             |
| **CumulatedSeepN**                               | Total nitrogen lost by seepage                                            | kg N ha⁻¹             |
| **NitrateLossSeepage**                           | Nitrate leached via seepage                                               | kg N ha⁻¹             |
| **CumulatedMineralizedP**                        | Cumulative mineralized phosphorus from soil organic matter                | kg P ha⁻¹             |
| **WSEEP\_SUM**                                   | Water lost via seepage (cumulative)                                       | mm                    |
| **soilwater\_fc\_global**                        | Soil water content at field capacity (global scale)                       | mm                    |
| **soilwater\_sat\_global**                       | Soil water content at saturation (global scale)                           | mm                    |
| **soilwater\_fc\_1**                             | Soil water at field capacity (topsoil layer)                              | mm                    |
| **soilwater\_sat\_1–4**                          | Soil water at saturation for layers 1–4                                   | mm                    |
| **carbon\_1–5**                                  | Soil organic carbon content for depth layers                              | % or g C kg⁻¹ soil    |
| **soil\_quality\_rating**                        | Soil quality index derived from soil properties                           | categorical / score   |


## Multicollinearity analysis
This function helps identify and reduce **multicollinearity** in a dataset by detecting highly correlated features and retaining only a representative subset. This is especially useful before running machine learning models, where correlated features can inflate variance and reduce model interpretability.

**Purpose**

* Detect **pairs of features** with high correlation.
* Drop redundant features while keeping one representative feature from each correlated group.
* Provide a reduced set of features that are less redundant and more informative.


In [ ]:
# Function to identify feature correlation
def get_reduced_correlated_features(df, threshold=0.8):
    """
    Identifies correlated feature pairs and selects a reduced set of features
    by keeping only one feature from each highly correlated group.

    Parameters:
        df (pd.DataFrame): The input DataFrame.
        threshold (float): Correlation coefficient threshold (default: 0.8).

    Returns:
        kept_features (List[str]): Features to keep (non-redundant).
        dropped_features (List[str]): Features identified as redundant and dropped.
        correlated_pairs (List[Tuple[str, str, float]]): Correlated feature pairs above threshold.
    """
    corr_matrix = df.corr(numeric_only=True)
    correlated_pairs = []
    to_drop = set()
    already_in_group = set()

    for i in range(len(corr_matrix.columns)):
        for j in range(i + 1, len(corr_matrix.columns)):
            col1 = corr_matrix.columns[i]
            col2 = corr_matrix.columns[j]
            corr_value = corr_matrix.iloc[i, j]
            if abs(corr_value) >= threshold:
                correlated_pairs.append((col1, col2, corr_value))
                # Drop col2 if col1 hasn't already been marked to drop
                if col1 not in to_drop and col2 not in already_in_group:
                    to_drop.add(col2)
                    already_in_group.add(col1)
                    already_in_group.add(col2)

    all_features = set(df.select_dtypes(include='number').columns)
    kept_features = sorted(list(all_features - to_drop))
    dropped_features = sorted(list(to_drop))

    return kept_features, dropped_features, correlated_pairs

## Splitting data into training and testing sets by year
This code splits the dataset into training and testing sets based on the number of unique `Harvest_Year` values. Instead of randomly splitting rows, it ensures the split happens chronologically (earlier years for training, later years for testing).

In [ ]:
# Check the number of years
n_train = int(data['Harvest_Year'].nunique() * 0.7)
n_test = data['Harvest_Year'].nunique() - n_train

train_years = sorted(data['Harvest_Year'].unique())[:n_train]
test_years = sorted(data['Harvest_Year'].unique())[-n_test:]

train_data = data[data['Harvest_Year'].isin(train_years)]
test_data = data[data['Harvest_Year'].isin(test_years)]

print(f'Train data shape: {train_data.shape}\nTest data shape: {test_data.shape}')
print('Years in the training data:', f'{min(train_years)}-{max(train_years)}')
print('Years in the testing data:', f'{min(test_years)}-{max(test_years)}')


## **Expanding window temporal cross validation**

This function implements a **temporal cross-validation** approach where the model is trained on an **expanding window of past years** and tested on the following year(s).

**How It Works**

1. Sort all unique years in the dataset.
2. For each fold:

   * **Training set**: all years from the earliest year up to a given year.
   * **Testing set**: the next `test_years` immediately following the training period.
3. Returns a list of **train/test index tuples** for each fold.


**Parameters**

* `data` (`pd.DataFrame`): Dataset with a year column.
* `year_col` (`str`): Column name representing the year.
* `min_train_years` (`int`): Minimum years to start training.
* `test_years` (`int`): Number of years to use for testing in each fold.

**Returns**

* `splits` (`list` of tuples): Each tuple contains `(train_indices, test_indices)` for one fold.

**Advantages**

* Keeps **temporal order** intact (no data leakage).
* Simulates real-world forecasting: train on past, test on future.
* Simple, easy-to-use for **time-series or seasonal data**


In [ ]:
def expanding_window_temporal_cv(
    data,
    year_col,
    min_train_years=5,
    test_years=1
):
    """
    Expanding Window Temporal Cross-Validation

    Splits the data into training and testing sets based on years using an expanding window.
    - Training set: all years up to a given year
    - Testing set: the immediately following `test_years`
    
    This is purely temporal CV; no locations are left out.

    Parameters:
    ----------
    data : pd.DataFrame
        Input dataframe containing a year column.
    year_col : str
        Column name representing the year (int or datetime).
    min_train_years : int, default=5
        Minimum number of years to begin training.
    test_years : int, default=1
        Number of years to use for testing in each fold.

    Returns:
    -------
    splits : list of (train_indices, test_indices)
        List of train/test index tuples for each fold.
    """
    splits = []
    all_years = sorted(data[year_col].unique())

    print("\nExpanding Window Temporal Cross-Validation\n" + "-" * 60)

    for end_train_idx in range(min_train_years, len(all_years) - test_years):
        train_years = all_years[:end_train_idx]
        test_years_range = all_years[end_train_idx:end_train_idx + test_years]

        train_idx = data[data[year_col].isin(train_years)].index.tolist()
        test_idx = data[data[year_col].isin(test_years_range)].index.tolist()

        splits.append((train_idx, test_idx))
        print(f"Train: {train_years[0]}-{train_years[-1]} | Test: {test_years_range[0]}-{test_years_range[-1]} "
              f"({len(train_idx)} train, {len(test_idx)} test)")

    return splits

In [ ]:
# Extract the group index
group_index = expanding_window_temporal_cv(
    train_data, 
    year_col='Harvest_Year',
    min_train_years=19,
    test_years=6
)

# Define the columns to be droppend from the data before training
cols_to_be_dropped = ["NUTS_ID", "Harvest_Year", "crop"]

## Feature scaling
This function standardizes the features of a training and testing dataset using the statistics from the training set only. This is a standard step in machine learning to make features comparable and improve model performance.

In [ ]:
# Function to standardize a train and test dataframe without considering NaN values
def standardize_train_test(train_df, test_df):
    """
    Standardizes train and test DataFrames based on train data statistics.
    
    Parameters:
    - train_df (pd.DataFrame): Training dataset
    - test_df (pd.DataFrame): Testing dataset
    
    Returns:
    - standardized_train (pd.DataFrame): Standardized training data
    - standardized_test (pd.DataFrame): Standardized test data
    """
    mean = train_df.mean(skipna=True)
    std = train_df.std(skipna=True)

    standardized_train = (train_df - mean) / std
    standardized_test = (test_df - mean) / std

    return standardized_train, standardized_test

## Model optimization using Optuna

In [ ]:
# Drop the highly correlated features from the data
kept_features, dropped_features, correlated_pairs = get_reduced_correlated_features(
    train_data.drop(columns=['NUTS_ID', 'Harvest_Year', 'crop', 'Yield_Obs']),
    threshold=0.90
)

print('Number of highly correlated features:', len(dropped_features))

# Drop the highly correlated features
train_data = train_data.drop(columns=dropped_features)
test_data = test_data.drop(columns=dropped_features)
print(train_data.shape, test_data.shape)
train_data.head()

### Optuna Objective function for XGBoost hyperparameter tuning

This function is used with **Optuna** to automatically search for the best hyperparameters of an `XGBRegressor` using **cross-validation**.

1. **Define hyperparameter search space**

   * `max_depth`: Maximum depth of each tree (shallow trees reduce overfitting).
   * `learning_rate`: Step size shrinkage for boosting.
   * `n_estimators`: Number of trees (boosting rounds).
   * `min_child_weight`: Minimum sum of instance weight per leaf.
   * `gamma`: Minimum loss reduction to make a split (regularization).
   * `subsample`: Fraction of rows to sample per tree.
   * `colsample_bytree`: Fraction of columns to sample per tree.
   * `reg_alpha` & `reg_lambda`: L1 and L2 regularization terms.

2. **Initialize XGBoost model**

   * Using GPU (`device="cuda"`) for faster training.
   * `tree_method="hist"` for optimized histogram-based training.

3. **Cross-validation loop**

   * Iterates over `group_index` containing predefined train/test splits (folds).
   * For each fold:

     1. Split train and test data.
     2. Drop unnecessary columns and target column from features.
     3. Standardize features using the `standardize_train_test` function.
     4. Train the model on training data.
     5. Predict on the test set.
     6. Calculate **RMSE** (Root Mean Squared Error) for the fold.

4. **Compute average RMSE**

   * Average RMSE across all folds is returned as the **objective score** for Optuna to minimize.

**Benefits**

* Automates hyperparameter tuning for XGBoost.
* Uses cross-validation to avoid overfitting to a single train/test split.
* Standardization ensures numerical stability across features.
* GPU acceleration speeds up the search.

In [ ]:
def objective(trial):
    """
    Optuna objective function for tuning XGBoost hyperparameters.
    
    Uses cross-validation to evaluate the average RMSE over predefined folds.
    """
    # Define the hyperparameter search space
    params = {
        "max_depth": trial.suggest_int("max_depth", 2, 6),                 # Tree depth
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.005, 0.05),  # Learning rate
        "n_estimators": trial.suggest_int("n_estimators", 100, 500),       # Number of trees
        "min_child_weight": trial.suggest_int("min_child_weight", 10, 50), # Minimum sum of instance weight
        "gamma": trial.suggest_loguniform("gamma", 0.5, 10),               # Regularization term for splits
        "subsample": trial.suggest_uniform("subsample", 0.5, 0.8),         # Row subsampling
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.5, 0.8), # Column subsampling
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1, 50),         # L1 regularization
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1, 50),       # L2 regularization
    }

    # Initialize the XGBoost model
    model = XGBRegressor(
        **params, 
        tree_method="hist", 
        device="cuda", 
        n_jobs=-1, 
        random_state=42
    )

    fold_losses = []

    # Cross-validation loop
    for _, (train_idx, test_idx) in enumerate(group_index):
        # Split features and target
        X_train = train_data.loc[train_idx].drop(cols_to_be_dropped + ['Yield_Obs'], axis=1)
        y_train = train_data.loc[train_idx]['Yield_Obs']

        X_test = train_data.loc[test_idx].drop(cols_to_be_dropped + ['Yield_Obs'], axis=1)
        y_test = train_data.loc[test_idx]['Yield_Obs']

        # Standardize the train and test sets
        X_train_scaled, X_test_scaled = standardize_train_test(X_train, X_test)

        # Fit the model and predict
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)

        # Compute RMSE for the fold
        fold_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        fold_losses.append(fold_rmse)

    # Return average RMSE across folds
    return np.mean(fold_losses)


In [ ]:
sampler = TPESampler(seed=42)
study = optuna.create_study(study_name="xgboost", sampler=sampler, direction="minimize")

study.optimize(objective, n_trials=100, show_progress_bar=True)

## Fitting the final model

In [ ]:
# Get the best hyperparameters
best_params = study.best_params

# Train the final model
X_train_info = train_data[cols_to_be_dropped + ['Yield_Obs']]
X_train = train_data.drop(cols_to_be_dropped + ['Yield_Obs'], axis=1)
y_train = train_data['Yield_Obs']

X_test_info = test_data[cols_to_be_dropped + ['Yield_Obs']]
X_test = test_data.drop(cols_to_be_dropped + ['Yield_Obs'], axis=1)
y_test = test_data['Yield_Obs']

# Standardize the data
X_train_scaled, X_test_scaled = standardize_train_test(X_train, X_test)

# Create a model instance
model = XGBRegressor(**best_params, tree_method="hist", predictor="gpu_predictor", device="cuda", random_state=42)

# Fit the model
model.fit(X_train_scaled, y_train)

# Predict on the test set and train set
y_pred_test = model.predict(X_test_scaled)
y_pred_train = model.predict(X_train_scaled)

# Update the preds_df
test_preds = X_test_info.copy()
train_preds = X_train_info.copy()

test_preds['Yield_Preds'] = y_pred_test
train_preds['Yield_Preds'] = y_pred_train

print(train_preds.shape, test_preds.shape)
test_preds.head()

In [ ]:
# Read the predicted data from both the scenario
pred_hybrid = test_preds.copy().drop('crop', axis=1)
pred_hybrid.rename(columns={'Yield_Obs': 'Yield (Obs)', 'Yield_Preds': 'Yield (Hybrid)'}, inplace=True)

# Add the result from the Simulated yield
sim_yield_df = data[['NUTS_ID', 'Harvest_Year', 'Yield_t_ha']].groupby(by=['NUTS_ID', 'Harvest_Year']).mean().reset_index()
sim_yield_df.rename(columns={'Yield_t_ha': 'Yield (PBM)'}, inplace=True)

global_preds = pd.merge(left=sim_yield_df, right=pred_hybrid,
                        on=['NUTS_ID', 'Harvest_Year'], how='inner')

global_preds = global_preds[['NUTS_ID', 'Harvest_Year', 'Yield (Obs)', 'Yield (PBM)', 'Yield (Hybrid)']]
print(global_preds.shape)
global_preds.head()

## Upscaling the model from NUTS3 to pixel scale

In [ ]:
# Define the datapath
data_path = os.path.join(WORK_DIR, 'data', "data_1km.csv")
data_1km = pd.read_csv(data_path)

data_1km_info = data_1km[['Location', 'NUTS_ID', 'Harvest_Year', 'Yield_t_ha', 'geometry']]
data_1km_info.rename(columns={'Yield_t_ha': 'Yield (PBM)'}, inplace=True)

# Drop all the correlated and unnecessary features
data_1km = data_1km[X_train_scaled.columns]

print(data_1km.shape)
data_1km.head()

In [ ]:
# Standardized the data
_, data_1km_scaled = standardize_train_test(X_train, data_1km)

# Predict the yield
data_1km_info['Yield (Hybrid-1sqkm)'] = model.predict(data_1km_scaled)
print(data_1km_info.shape)
data_1km_info.head()

In [ ]:
# Groupby the data and compare with observed yield
data_1km_grouped = data_1km_info.drop(columns=['Location', 'geometry'])\
    .groupby(by=['NUTS_ID', 'Harvest_Year'])\
    .mean()\
    .reset_index()
    
data_1km_grouped = pd.merge(
    left=data_1km_grouped, 
    right=global_preds[['NUTS_ID', 'Harvest_Year', 'Yield (Obs)']],
    how='inner',
    on=['NUTS_ID', 'Harvest_Year']
)

print(data_1km_grouped.shape)
data_1km_grouped.head()

In [ ]:
global_preds = pd.merge(
    left=global_preds, 
    right=data_1km_grouped[['NUTS_ID', 'Harvest_Year', 'Yield (Hybrid-1sqkm)']],
    on=['NUTS_ID', 'Harvest_Year'],
    how='inner'
)
print(global_preds.shape)
global_preds.head()

## Final evaluation

In [ ]:
# Melt to long format for ease of plotting
df_melted = global_preds.melt(id_vars=["NUTS_ID", "Harvest_Year", "Yield (Obs)"],
                    value_vars=['Yield (PBM)', 'Yield (Hybrid)', 'Yield (Hybrid-1sqkm)'],
                    var_name="Type", value_name="Predicted")

# Calculate MAPE
df_melted["MAPE"] = np.abs((df_melted["Yield (Obs)"] - df_melted["Predicted"]) / df_melted["Yield (Obs)"]) * 100
print(df_melted.shape)
df_melted.head()

In [ ]:
# Compute mean MAPE per model type
mean_mapes = df_melted.groupby("Type")["MAPE"].mean()

# Define colors to match barplot
colors = {
    "Yield (PBM)": "#ffa600",
    "Yield (Hybrid)": "#ff6361",
    "Yield (Hybrid-1sqkm)": "#58508d"
}

# Plot
plt.figure(figsize=(12, 6), dpi=300)

sns.barplot(
    data=df_melted,
    x="Harvest_Year", y="MAPE", hue="Type",
    ci="sd", capsize=0.2, errwidth=1.2,
    palette=[colors["Yield (PBM)"], colors["Yield (Hybrid)"], colors["Yield (Hybrid-1sqkm)"]],
    edgecolor='k'
)

# Add horizontal mean lines
for model_type, mean_val in mean_mapes.items():
    plt.axhline(y=mean_val, color=colors[model_type], linestyle='--', linewidth=1.5)

# Save handles and labels from legend
handles, labels = plt.gca().get_legend_handles_labels()

# Custom label mapping
label_map = {
    "Yield (PBM)": f"PBM ({mean_mapes['Yield (PBM)'].round(2)}%)",
    "Yield (Hybrid)": f"Hybrid ({mean_mapes['Yield (Hybrid)'].round(2)}%)",
    "Yield (Hybrid-1sqkm)": f"Hybrid (1 km²) ({mean_mapes['Yield (Hybrid-1sqkm)'].round(2)}%)"
}

# Apply new labels to legend
plt.legend(handles=handles, labels=[label_map[l] for l in labels], title="", loc='upper left', frameon=False)

# Final touches
plt.title("Year-wise MAPE of Yield Predictions (with Standard Deviation)")
plt.xlabel("Test Year")
plt.ylabel("MAPE (%)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Convert the prediction Into a raster

In [ ]:
states = ['Berlin', 'Brandenburg']

# Read the coordinates and convert it into geodataframe
coords_df = pd.read_csv(os.path.join(WORK_DIR, 'data', 'location.csv'))
coords_df["geometry"] = coords_df["geometry"].apply(wkt.loads)
coords_gdf = gpd.GeoDataFrame(coords_df, geometry='geometry', crs='EPSG:3857')
coords_gdf.to_crs('EPSG:31467', inplace=True)
coords_gdf = coords_gdf[coords_gdf['STATE_NAME'].isin(states)]
coords_gdf.rename(columns={'Cell_ID': 'Location'}, inplace=True)
print(coords_gdf.shape)
coords_gdf.head()

In [ ]:
data_1km_info_gdf = pd.merge(
    left=data_1km_info.drop(columns=['geometry']), 
    right=coords_gdf[['Location', 'geometry']],
    on=['Location'],
    how='inner'
)

data_1km_info_gdf['geometry'] = data_1km_info_gdf['geometry'].apply(
    lambda x: wkt.loads(x) if isinstance(x, str) else x
)

data_1km_info_gdf = gpd.GeoDataFrame(data_1km_info_gdf, crs='EPSG:31467')
data_1km_info_gdf.rename(columns={"Yield (Hybrid-1sqkm)": 'Yield (Hybrid)'}, inplace=True)
print(data_1km_info_gdf.shape)
data_1km_info_gdf.head()

In [ ]:
unique_years = sorted(data_1km_info_gdf["Harvest_Year"].unique())

# Dictionary to hold lists of DataArrays for each measurement
measurements = list(data_1km_info_gdf.columns[3:-1])
measurement_arrays = {m: list() for m in measurements}

# Loop through each year and rasterize
for year in tqdm(unique_years):
    gdf_year = data_1km_info_gdf[data_1km_info_gdf["Harvest_Year"] == year]

    cube = make_geocube(
        vector_data=gdf_year,
        measurements=measurements,
        resolution=(-1000, 1000),
        fill=np.nan
    )

    for measurement in measurement_arrays:
        # Expand along time dimension and add to list
        data_array = cube[measurement].expand_dims(year=[year])
        measurement_arrays[measurement].append(data_array)

# Combine each measurement into one DataArray (over years)
combined_vars = {
    var_name: xr.concat(arr_list, dim="year")
    for var_name, arr_list in measurement_arrays.items()
}

# Create a Dataset from all variables
combined_dataset = xr.Dataset(combined_vars)

# Save to NetCDF
combined_dataset.to_netcdf(os.path.join(WORK_DIR, 'output', 'prediction.nc'))

In [ ]:
# Load dataset
ds = xr.open_dataset(os.path.join(WORK_DIR, 'output', 'prediction.nc'))

var_name = "Yield (Hybrid)"

# Compute mean and std over spatial dims (x, y) for each year
mean_vals = ds[var_name].mean(dim=["x", "y"])
std_vals = ds[var_name].std(dim=["x", "y"])

years = mean_vals.year.values

plt.figure(figsize=(8, 5))
plt.plot(years, mean_vals, color='blue', marker='o', label=var_name)
plt.fill_between(years,
                 mean_vals - std_vals,
                 mean_vals + std_vals,
                 color='blue',
                 alpha=0.1)

plt.title(f"Mean {var_name} over time with standard deviation")
plt.xlabel("Year")
plt.ylabel(f"{var_name}")
plt.grid(True)
plt.legend()
plt.show()

## Apply the model on CMIP future scenarios

In [ ]:
# Define scenario name
scenario_name = 'MPI-ESM1-2-HR_ssp585'

# Define the datapath
data_path = os.path.join(WORK_DIR, 'data', f"data_1km_{scenario_name}.csv")
scenario_data = pd.read_csv(data_path)
scenario_data = scenario_data[scenario_data['Harvest_Year']>2022].reset_index(drop=True)
future = scenario_data[X_train.columns]
scenario_data_info = scenario_data[['Location', 'NUTS_ID', 'Harvest_Year', 'Yield_t_ha']]
scenario_data_info.rename(columns={'Yield_t_ha': 'Yield (PBM)'}, inplace=True)

X_train_scaled, future_scaled = standardize_train_test(X_train, future)

# Predict the yield
scenario_data_info ['Yield (Hybrid)'] = model.predict(future_scaled)

scenario_data_info = pd.merge(
    left=scenario_data_info,
    right=data_1km_info_gdf[['Location', 'geometry']].drop_duplicates(subset='Location'),
    on=['Location'],
    how='left'
)

scenario_data_info = gpd.GeoDataFrame(scenario_data_info)
print(scenario_data_info.shape)
scenario_data_info.head()

In [ ]:
unique_years = sorted(scenario_data_info["Harvest_Year"].unique())

# Dictionary to hold lists of DataArrays for each measurement
measurements = list(data_1km_info_gdf.columns[3:-1])
measurement_arrays = {m: list() for m in measurements}

# Loop through each year and rasterize
for year in tqdm(unique_years):
    gdf_year = scenario_data_info[scenario_data_info["Harvest_Year"] == year]

    cube = make_geocube(
        vector_data=gdf_year,
        measurements=measurements,
        resolution=(-1000, 1000),
        fill=np.nan
    )

    for measurement in measurement_arrays:
        # Expand along time dimension and add to list
        data_array = cube[measurement].expand_dims(year=[year])
        measurement_arrays[measurement].append(data_array)

# Combine each measurement into one DataArray (over years)
combined_vars = {
    var_name: xr.concat(arr_list, dim="year")
    for var_name, arr_list in measurement_arrays.items()
}

# Create a Dataset from all variables
combined_dataset = xr.Dataset(combined_vars)

# Save to NetCDF
combined_dataset.to_netcdf(os.path.join(WORK_DIR, 'output', f"prediction_{scenario_name}.nc"))

## Final plots

In [ ]:
# Load and prepare NetCDF datasets
hist_ds = xr.open_dataset(os.path.join(WORK_DIR, 'output', 'prediction.nc'))
futu_ds = xr.open_dataset(os.path.join(WORK_DIR, 'output', f'prediction_{scenario_name}.nc'))

var_name = "Yield (PBM)"
hist_var = hist_ds[var_name]
futu_var = futu_ds[var_name]

# Assign CRS and reproject both datasets to EPSG:4326
hist_var = hist_var.rio.write_crs("EPSG:31467").rio.reproject("EPSG:4326")
futu_var = futu_var.rio.write_crs("EPSG:31467").rio.reproject("EPSG:4326")

# Concatenate datasets along 'year'
combined_var = xr.concat([hist_var, futu_var], dim='year')
combined_var = combined_var.sortby('year')

# Define 15-year intervals
intervals = [(1980, 2000), (2001, 2020), (2021, 2040), (2041, 2060), (2061, 2080), (2081, 2100)]

# Load shapefile (Brandenburg)
gdf = gpd.read_file(os.path.join(WORK_DIR, 'data', 'brandenburg.gpkg'))

# Compute 15-year means
mean_maps = []
for start, end in intervals:
    mean = combined_var.sel(year=slice(start, end)).mean(dim='year')
    mean_maps.append((f"{start}-{end}", mean))

# Determine common color scale
vmin = min([m[1].min().item() for m in mean_maps])
vmax = max([m[1].max().item() for m in mean_maps])

# Plot subplots
n_maps = len(mean_maps)
ncols = 3
nrows = (n_maps + ncols - 1) // ncols

fig, axes = plt.subplots(nrows, ncols, figsize=(5*ncols, 5*nrows),
                         subplot_kw={'projection': ccrs.PlateCarree()})

axes = axes.flatten()

# Plot subplots
for i, (title, d) in enumerate(mean_maps):
    ax = axes[i]
    gdf.boundary.plot(ax=ax, edgecolor='black', linewidth=0.8, transform=ccrs.PlateCarree(), zorder=3)
    im = d.plot(ax=ax, transform=ccrs.PlateCarree(), cmap='RdYlGn',
                   vmin=vmin, vmax=vmax, add_colorbar=False, zorder=2)
    # gdf.boundary.plot(ax=ax, edgecolor='black', facecolor='#eaeaea', linewidth=0.8, transform=ccrs.PlateCarree(), zorder=1)
    ax.set_title(f"Mean Yield: {float(d.mean()):.2f} | Year: {title}", fontsize=14)

# Remove unused axes if any
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

# Add shared colorbar on the right side
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # [left, bottom, width, height]
cbar = fig.colorbar(im, cax=cbar_ax, label=f'{var_name} (t/ha)')
cbar.set_label(f'{var_name} (t/ha)', fontsize=14)
cbar.ax.tick_params(labelsize=12) 

plt.tight_layout(rect=[0, 0, 0.9, 1])
plt.show()

In [ ]:
var_names = ['Yield (PBM)', 'Yield (Hybrid)']
yearly_timeseries = {}

for var in var_names:
    hist_var = hist_ds[var]
    futu_var = futu_ds[var]
    
    combined_var = xr.concat([hist_var, futu_var], dim='year')
    combined_var = combined_var.sortby('year')
    yearly_mean = combined_var.mean(dim=['y', 'x'])
    yearly_std = combined_var.std(dim=['y', 'x'])
    yearly_df = pd.DataFrame(
        {'Year': yearly_mean.year, 'Yield (Mean)': yearly_mean, 'Yield (Std)': yearly_std}
    )
    yearly_timeseries[var] = yearly_df

In [ ]:
plt.figure(figsize=(14, 4), dpi=150)
for i, (t, df) in enumerate(yearly_timeseries.items()):
    sns.lineplot(data=df, x='Year', y='Yield (Mean)', marker='o', markersize=4, label=t)
    
    # Plot the fill between mean ± std
    plt.fill_between(
        df['Year'],
        df['Yield (Mean)'] - df['Yield (Std)'],
        df['Yield (Mean)'] + df['Yield (Std)'],
        alpha=0.3
    )

sns.lineplot(data=data, x='Harvest_Year', y='Yield_Obs', marker='o', markersize=4, label='Yield (Observed)') 
plt.grid()
plt.ylabel('Yield (t/ha)');

## **SHAP analysis**

In [ ]:
# Define the SHAP explainer
explainer = shap.TreeExplainer(model)

# Calculate SHAP values for the entire test set
shap_values = explainer.shap_values(X_test_scaled)

# summary plot (global importance)
shap.summary_plot(shap_values, X_test_scaled)

In [ ]:
# Define variables and names
selected_variables = {
    'Minimum Temperature (Pre-planting window)': 'Tmin_avg_p0',
    'Radiation (Vegetative phase)': 'Radiation_avg_p2',
    'Minimum Temperature (Harvest window)': 'Tmin_avg_p5',
    'Precipitation (Planting window)': 'Rain_avg_p1',
    'Above Ground Biomass (Vegetative phase)': 'AGBiomass_t_ha_max_p2',
    'Total Soil-Available Water (Vegetative phase)': 'TotalSoilAvailWaterContent_avg_p2'
}

# Set up plot grid
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(18, 8))
axes = axes.flatten()

# Set consistent color normalization range across all plots
shap_min, shap_max = shap_values.min(), shap_values.max()

# Loop through variables
for i, (name, var) in enumerate(selected_variables.items()):
    shap_vals = shap_values[:, X_test.columns.get_loc(var)]
    feat_vals = X_test[var]
    
    # DataFrame for plotting
    df = pd.DataFrame({var: feat_vals, 'SHAP': shap_vals})
    
    # LOWESS smoothing
    smoothed = lowess(endog=df['SHAP'], exog=df[var], frac=0.2)
    
    # Scatter plot with SHAP coloring
    sns.scatterplot(
        data=df,
        x=var,
        y='SHAP',
        hue='SHAP',
        palette='PiYG',
        hue_norm=(shap_min, shap_max),
        ax=axes[i],
        s=50,
        alpha=0.7,
        edgecolor='k',
        linewidth=0.5,
        legend=False
    )
    
    # Smoothed trend line
    axes[i].plot(smoothed[:, 0], smoothed[:, 1], color='red', lw=2, label='Smoothed trend')
    
    # Horizontal zero line
    axes[i].axhline(0, color='black', linestyle='--', linewidth=1)
    
    # Labels
    axes[i].set_xlabel(name, fontsize=11)
    axes[i].set_ylabel('SHAP value', fontsize=11)
    axes[i].tick_params(labelsize=10)
    axes[i].set_title(name, fontsize=12, fontweight='bold')

# Layout and title
plt.tight_layout()
plt.suptitle("SHAP Dependency Plots for Key Predictors", fontsize=16, y=1.02)
plt.show()